In [1]:
import os
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import Chroma
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import Tool
from langchain_ollama import OllamaEmbeddings
from langchain_community.llms import Ollama

In [2]:
files = [
    {
        "name": "American Football",
        "path": r"C:\Users\yahoo\OneDrive\Desktop\project sample ready\American Football.pdf",
    },
    {
        "name": "British football",
        "path": r"C:\Users\yahoo\OneDrive\Desktop\project sample ready\British Football.pdf",
    },
]

In [3]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

for file in files:
    loader = PyPDFLoader(file["path"])
    pages = loader.load_and_split()
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    embeddings = OllamaEmbeddings(model="gemma")
    retriever = FAISS.from_documents(docs, embeddings).as_retriever()

In [4]:
llm = Ollama(model="gemma")


In [5]:
#Contextualize Question Prompt:
# Contextualize question prompt
# This system prompt helps the AI understand that it should reformulate the question
# based on the chat history to make it a standalone question
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

In [6]:
# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [7]:
# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [8]:
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "only the provided documents to answer the question. If you don't know the answer based on the provided documents, just say 'I don't know'. "
    "Do not attempt to search the internet or use external sources. Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)


In [9]:
# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [10]:
# Create a chain to combine documents for question answering
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

In [11]:
rag_chain = create_retrieval_chain(
    history_aware_retriever, question_answer_chain)

In [12]:

from langchain_core.prompts import PromptTemplate

react_docstore_prompt = """
You are an intelligent agent designed to process and understand multiple documents in the {tools}. 
Before providing an answer, you need to review all the provided documents in the {tools}
and then synthesize the information to give a comprehensive and accurate response.If the information is not in the 
{tools} you say dont know.

Use the following format:

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat 3 times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""

# Wrap the string in a PromptTemplate
react_docstore_prompt_template = PromptTemplate.from_template(react_docstore_prompt)


In [13]:
from langchain_core.tools import Tool

def answer_question_tool(input, **kwargs):
    print(f"Received input: {input}")
    response = rag_chain.invoke({"input": input, "chat_history": kwargs.get("chat_history", [])})
    print(f"Response from rag_chain: {response}")
    return response

tools = [
    Tool(
        name="Answer Question",
        func=answer_question_tool,
        description="Useful for answering questions using provided documents. Will not attempt to answer anything outside the documents.",
    )
]


In [14]:
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_docstore_prompt_template,  # Correct template object
)


In [15]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, handle_parsing_errors=True, verbose=True,
)

In [16]:
def test_agent_functionality(agent_executor):
    queries = [
        "What is the capital of France?",
        "exit"
    ]

    chat_history = []
    for query in queries:
        if query.lower() == "exit":
            break
        
        print(f"Processing query: {query}")
        try:
            response = agent_executor.invoke({"input": query, "chat_history": chat_history})
            print(f"AI: {response['output']}")
            
            chat_history.append(HumanMessage(content=query))
            chat_history.append(AIMessage(content=response["output"]))
        except Exception as e:
            print(f"An error occurred: {e}")

# Run the test
test_agent_functionality(agent_executor)


Processing query: What is the capital of France?


> Entering new AgentExecutor chain...
Thought: The documents contain information about the capital of France.

Action: Answer Question

Action Input: "What is the capital of France?"Received input: What is the capital of France?
Response from rag_chain: {'input': 'What is the capital of France?', 'chat_history': [], 'context': [Document(metadata={'source': 'C:\\Users\\yahoo\\OneDrive\\Desktop\\project sample ready\\British Football.pdf', 'page': 0}, page_content="British Football  \nBritish football, known as soccer in the United States, has roots tracing back to \nmedieval England and was formalized into its modern form in the 19th century. \nPlayed with a round ball on a rectangular field with goals at each end, the game \nfeatures teams of 11 players who aim to score by getting the ball into the \nopponent's goal. Soccer is known for its emphasis on skill and strategy rather than \nphysical collisions. The game is divided into two 4

In [17]:
def test_agent_functionality(agent_executor):
    queries = [
        "Explain British football",
        "exit"
    ]

    chat_history = []
    for query in queries:
        if query.lower() == "exit":
            break
        
        print(f"Processing query: {query}")
        try:
            response = agent_executor.invoke({"input": query, "chat_history": chat_history})
            print(f"AI: {response['output']}")
            
            chat_history.append(HumanMessage(content=query))
            chat_history.append(AIMessage(content=response["output"]))
        except Exception as e:
            print(f"An error occurred: {e}")

# Run the test
test_agent_functionality(agent_executor)


Processing query: Explain British football


> Entering new AgentExecutor chain...
Action: Answer Question
Action Input: Explain British footballReceived input: Explain British football
Response from rag_chain: {'input': 'Explain British football', 'chat_history': [], 'context': [Document(metadata={'source': 'C:\\Users\\yahoo\\OneDrive\\Desktop\\project sample ready\\British Football.pdf', 'page': 0}, page_content="British Football  \nBritish football, known as soccer in the United States, has roots tracing back to \nmedieval England and was formalized into its modern form in the 19th century. \nPlayed with a round ball on a rectangular field with goals at each end, the game \nfeatures teams of 11 players who aim to score by getting the ball into the \nopponent's goal. Soccer is known for its emphasis on skill and strategy rather than \nphysical collisions. The game is divided into two 45 -minute halves, with limited \nphysical contact allowed. It is  the world’s most popular sport, wi

In [16]:
def test_agent_functionality(agent_executor):
    queries = [
        "Compare British football and American football",
        "exit"
    ]

    chat_history = []
    for query in queries:
        if query.lower() == "exit":
            break
        
        print(f"Processing query: {query}")
        try:
            response = agent_executor.invoke({"input": query, "chat_history": chat_history})
            print(f"AI: {response['output']}")
            
            chat_history.append(HumanMessage(content=query))
            chat_history.append(AIMessage(content=response["output"]))
        except Exception as e:
            print(f"An error occurred: {e}")

# Run the test
test_agent_functionality(agent_executor)


Processing query: Compare British football and American football


> Entering new AgentExecutor chain...
Thought: I should review the provided documents to gather information about both British football and American football.

Action: Answer Question

Action Input: Documents related to British football and American footballReceived input: Documents related to British football and American football
Response from rag_chain: {'input': 'Documents related to British football and American football', 'chat_history': [], 'context': [Document(metadata={'source': 'C:\\Users\\yahoo\\OneDrive\\Desktop\\project sample ready\\British Football.pdf', 'page': 0}, page_content="British Football  \nBritish football, known as soccer in the United States, has roots tracing back to \nmedieval England and was formalized into its modern form in the 19th century. \nPlayed with a round ball on a rectangular field with goals at each end, the game \nfeatures teams of 11 players who aim to score by getting the bal

In [16]:
'''chat_history = []
while True:
    query = input("You: ")
    if query.lower() == "exit":
        break
    response = agent_executor.invoke(
        {"input": query, "chat_history": chat_history})
    print(f"AI: {response['output']}")

    # Update history
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=response["output"]))'''